<a href="https://colab.research.google.com/github/Signed-B/build-your-own-llm/blob/main/colab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers
# !pip install -q peft
!pip install -q accelerate
!pip install -q trl xformers wandb datasets einops gradio sentencepiece bitsandbytes

In [2]:
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, HfArgumentParser, TrainingArguments, pipeline, logging, TextStreamer
# from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
# import wandb
import os,torch, platform, gradio, warnings
from datasets import load_dataset
# from trl import SFTTrainer
# from huggingface_hub import notebook_login

In [3]:
def print_system_specs():
    # Check if CUDA is available
    is_cuda_available = torch.cuda.is_available()
    print("CUDA Available:", is_cuda_available)
# Get the number of available CUDA devices
    num_cuda_devices = torch.cuda.device_count()
    print("Number of CUDA devices:", num_cuda_devices)
    if is_cuda_available:
        for i in range(num_cuda_devices):
            # Get CUDA device properties
            device = torch.device('cuda', i)
            print(f"--- CUDA Device {i} ---")
            print("Name:", torch.cuda.get_device_name(i))
            print("Compute Capability:", torch.cuda.get_device_capability(i))
            print("Total Memory:", torch.cuda.get_device_properties(i).total_memory, "bytes")
    # Get CPU information
    print("--- CPU Information ---")
    print("Processor:", platform.processor())
    print("System:", platform.system(), platform.release())
    print("Python Version:", platform.python_version())
print_system_specs()

CUDA Available: True
Number of CUDA devices: 1
--- CUDA Device 0 ---
Name: Tesla T4
Compute Capability: (7, 5)
Total Memory: 15835660288 bytes
--- CPU Information ---
Processor: x86_64
System: Linux 6.1.58+
Python Version: 3.10.12


In [4]:
!pip install accelerate

In [5]:
# Pre trained model
model_name = "mosaicml/mpt-7b-instruct"

# Dataset name
dataset_name = "vicgalle/alpaca-gpt4"

In [6]:
model = transformers.AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map='cuda',
  torch_dtype=torch.bfloat16,
  trust_remote_code=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/376fbc91dcf911a7f8a82fc6359620478e92ca29/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/root/.cache/huggingface/modules/transformers_modules/mosaicml/mpt-7b-instruct/376fbc91dcf911a7f8a82fc6359620478e92ca29/configuration_mpt.py:141: UserWarning: If not usin

pytorch_model.bin.index.json:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

KeyboardInterrupt: 